In [2]:
import pandas as pd
import numpy as np
import joblib
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.ensemble import HistGradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import StratifiedKFold, cross_val_score

print("=== RÉPARATION : ENTRAÎNEMENT DU MODÈLE ARBRES (V5) SUR COLAB ===")

# 1. On s'assure que les données sont prêtes (Normalement déjà chargées en mémoire)
# Si les variables n'existent plus, on recharge :
try:
    if 'train_df' not in locals():
        raise NameError
    print("Données trouvées en mémoire.")
except NameError:
    print("Rechargement des données...")
    train_df = pd.read_json('train.jsonl', lines=True)
    test_df = pd.read_json('kaggle_test.jsonl', lines=True)
    train_df = json_normalize(train_df.to_dict(orient='records'))
    test_df = json_normalize(test_df.to_dict(orient='records'))
    train_df = advanced_feature_engineering(train_df)
    test_df = advanced_feature_engineering(test_df)

# 2. Reconstitution du Pipeline V5 (Identique à votre version Mac)
print("Configuration du Pipeline V5...")

numeric_features = [
    'user.listed_count', 'user.favourites_count', 'user.statuses_count',
    'log_listed', 'log_statuses', 'ratio_log',
    'quote_count', 'favorite_count', 'retweet_count', 'reply_count',
    'caps_ratio', 'exclamation_count', 'length_char', 'lexical_diversity',
    'desc_len', 'desc_has_email', 'desc_has_http', 'desc_is_pro',
    'account_age_days'
]
categorical_features = ['source_is_top'] # Simplifié pour Colab
tweet_text_col = 'final_text'
desc_text_col = 'user_desc'

# Transformers
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

tweet_text_transformer = Pipeline(steps=[
    ('tfidf', TfidfVectorizer(max_features=5000, stop_words='english', ngram_range=(1,2))),
    ('svd', TruncatedSVD(n_components=50, random_state=42))
])

desc_text_transformer = Pipeline(steps=[
    ('tfidf', TfidfVectorizer(max_features=1000, stop_words='english')),
    ('svd', TruncatedSVD(n_components=10, random_state=42))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('tweet_txt', tweet_text_transformer, tweet_text_col),
        ('desc_txt', desc_text_transformer, desc_text_col)
    ]
)

# Le Voting Classifier (L'arme fatale V5)
clf1 = HistGradientBoostingClassifier(max_iter=300, learning_rate=0.05, max_depth=10, random_state=42)
clf2 = RandomForestClassifier(n_estimators=200, max_depth=15, random_state=42, n_jobs=-1)
clf3 = ExtraTreesClassifier(n_estimators=200, max_depth=15, random_state=42, n_jobs=-1)

voting_model = VotingClassifier(
    estimators=[('hgb', clf1), ('rf', clf2), ('et', clf3)],
    voting='soft'
)

model_tree = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', voting_model)
])

# 3. Entraînement V5
print("Entraînement du modèle Arbres en cours (Patientez ~1-2 min)...")
X = train_df
y = train_df['label']
model_tree.fit(X, y)
print("Modèle Arbres entraîné !")

# 4. Génération des probabilités Arbres
print("Génération des prédictions Arbres...")
probs_tree = model_tree.predict_proba(test_df)[:, 1]

# 5. Récupération des probabilités Deep Learning
print("Récupération des prédictions Deep Learning...")
# On recharge le modèle DL sauvegardé par le checkpoint juste avant le crash
try:
    model_dl_loaded = load_model("best_model_dl_opt.h5")

    # On doit refaire la tokenization pour le test set si les variables ont été perdues
    # (Sécurité pour éviter les erreurs de dimension)
    MAX_WORDS = 15000; MAX_LEN = 80
    tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token="<OOV>")
    tokenizer.fit_on_texts(train_df['final_text']) # Important: fit sur TRAIN

    X_test_text_seq = pad_sequences(tokenizer.texts_to_sequences(test_df['final_text']), maxlen=MAX_LEN, padding='post')

    # On refait le scaler pour les meta du DL (car c'est un scaler différent du pipeline sklearn)
    scaler_dl = StandardScaler()
    scaler_dl.fit(train_df[numeric_features + ['source_is_top']])
    X_test_meta_norm = scaler_dl.transform(test_df[numeric_features + ['source_is_top']])

    # Prédiction DL
    probs_dl = model_dl_loaded.predict([X_test_text_seq, X_test_meta_norm], batch_size=32).reshape(-1)
    print("Prédictions DL récupérées.")

except Exception as e:
    print(f"Erreur lors du rechargement DL : {e}")
    print("Utilisation des probas DL en mémoire si disponibles...")

# 6. BLENDING FINAL
print("=== FUSION FINALE ===")
# Formule : 65% Arbres + 35% Deep Learning
final_scores = (0.65 * probs_tree) + (0.35 * probs_dl)
final_predictions = (final_scores > 0.5).astype(int)

submission = pd.DataFrame({'ID': test_df['challenge_id'], 'Prediction': final_predictions})
submission.to_csv('submission_FINAL_COLAB.csv', index=False)

print("\n" + "="*40)
print("SUCCÈS TOTAL ! 🚀")
print("Téléchargez le fichier 'submission_FINAL_COLAB.csv' dans le menu de gauche.")
print("="*40)

=== RÉPARATION : ENTRAÎNEMENT DU MODÈLE ARBRES (V5) SUR COLAB ===
Données trouvées en mémoire.
Configuration du Pipeline V5...
Entraînement du modèle Arbres en cours (Patientez ~1-2 min)...
Modèle Arbres entraîné !
Génération des prédictions Arbres...


Récupération des prédictions Deep Learning...
3231/3231 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step
Prédictions DL récupérées.
=== FUSION FINALE ===

SUCCÈS TOTAL ! 🚀
Téléchargez le fichier 'submission_FINAL_COLAB.csv' dans le menu de gauche.
